# Crate Optimisation

We simulate crate geometries for a **banana-fibre/HDPE produce crate** (600 × 400 mm), checking for **strength**, **stiffness**, and **handling feasibility**.

---

## 1. Goal

Simulate realistic returnable crate designs that:

- Don’t buckle or yield under stacking loads  
- Don’t sag more than 3 mm under produce  
- Weigh between 1.5 and 3.0 kg for ergonomic handling  
- Are manufacturable (minimum wall and rib sizes)

---

## 2. Parameters & Material Data

| Variable      | Meaning                            | Value               |
|---------------|-------------------------------------|---------------------|
| `L`, `W`      | Crate footprint                     | 0.60 m × 0.40 m     |
| `H_WALL`      | Wall height                         | 0.22 m              |
| `M_LOAD`      | Load per crate                      | 8.0 kg              |
| `STACK_MAX`   | Max stack height                    | 6 crates            |
| `g`           | Gravity                             | 9.81 m/s²           |
| `E`           | Young's modulus of material         | 1.2 × 10⁹ Pa        |
| `σ_y`         | Yield strength of material          | 18 × 10⁶ Pa         |
| `ρ`           | Density of foamed banana-HDPE       | 850 kg/m³           |
| `t_rib`       | Rib wall thickness (fixed)          | 3 mm                |

🔎 **Sources**:  
- Roark’s Formulas for Stress and Strain (bending formulas)  
- Euler's Column Buckling Formula  
- [Study of the Physical, Mechanical and Thermal Properties of Banana Fiber Reinforced HDPE Composites](https://doi.org/10.4236/msa.2020.114017)

---

## 3. Design Variables

| Variable    | Description                     | Range (mm)         |
|-------------|----------------------------------|---------------------|
| `t_skin`    | Bottom panel thickness          | 1.5 – 3.0           |
| `t_wall`    | Wall & corner post thickness    | 3.0 – 4.5           |
| `pitch`     | Rib spacing (center to center)  | 70 – 100            |
| `h_rib`     | Rib height                      | 15 – 24             |

---

## 4. Mechanical Model

Each configuration is tested using the physics below:

---

### 4.1 Bottom Panel Bending

**Assumption**: The panel acts like a simply supported beam between ribs (span = pitch `p`), under **uniform pressure** from produce weight.

#### Load per unit area:
The fruit weight is spread across the floor:

$$
q = \frac{M_{LOAD} \cdot g}{L \cdot W}
$$

#### Bending stress in the panel:

This is derived from the bending moment for a uniformly loaded beam:

$$
\sigma_\text{panel} = \frac{6 q p^2}{8 t_\text{skin}^2}
$$

Where:
- $( p $) = rib spacing (m) = panel span
- $( t_\text{skin} )$ = thickness of the panel

#### Mid-span deflection:

From elastic beam theory (Roark):

$$
\delta = \frac{5 q p^4}{384 E t_\text{skin}^3}
$$

✅ Passes if:

- $( \sigma_\text{panel} \le 0.85 \cdot \sigma_y )$ (avoid plastic deformation)  
- $( \delta \le 0.003 \, \text{m} )$ (looks flat, robot-safe)

---

### 4.2 Rib Compression & Buckling

Each rib supports part of the total stacking load:

#### Total vertical load from stacked crates:

$$
P_\text{stack} = (N - 1) \cdot M_{LOAD} \cdot g
$$

This is shared between all ribs (total number estimated from crate size and pitch).

#### Compressive stress in each rib:

$$
\sigma_\text{rib} = \frac{P_\text{stack} / n_\text{ribs}}{t_\text{rib} \cdot h_\text{rib}}
$$

#### Buckling limit (Euler's critical stress):

$$
\sigma_\text{Euler} = \frac{\pi^2 E}{(h_\text{rib} / t_\text{rib})^2}
$$

✅ Passes if:

- $( \sigma_\text{rib} \le 0.5 \cdot \sigma_y )$  
- $( \sigma_\text{rib} \le 0.8 \cdot \sigma_\text{Euler} )$  
(to avoid early buckling or over-compression)

---

### 4.3 Corner Post Compression

Each post carries **¼ of the total stacking weight**, including crate weight:

$$
\sigma_\text{post} = \frac{(N - 1)(M_{LOAD} + m_{crate})g}{4 \cdot t_{wall}^2}
$$

Where $( m_{crate} )$ is estimated mass.

✅ Passes if:

- $( \sigma_\text{post} \le 0.4 \cdot \sigma_y )$  
(40% long-term limit due to HDPE creep behavior)

---

### 4.4 Mass Check

Total crate mass is calculated as:

$$
m = \rho \cdot (V_{skin} + V_{ribs} + V_{walls} + V_{posts})
$$

Where:
- $( V_{skin} = L \cdot W \cdot t_{skin} )$
- $( V_{ribs} )$ from total rib length × cross section
- $( V_{walls} = 2(L + W) \cdot H_{WALL} \cdot t_{wall} )$
- $( V_{posts} = 4 \cdot H_{WALL} \cdot t_{wall}^2 )$

✅ Acceptable range:

$$
1.5 \le m \le 3.0 \, \text{kg}
$$

---

## 5. What the Output Table Shows

| Column     | Meaning                                           |
|------------|---------------------------------------------------|
| `ts`, `tw` | Bottom skin and wall thickness (mm)               |
| `pitch`    | Rib spacing (mm)                                  |
| `h_rib`    | Rib height (mm)                                   |
| `Mass`     | Crate mass (kg)                                   |
| `U_panel`  | Stress in panel / yield (%)                       |
| `U_rib`    | Rib compression stress / yield (%)                |
| `U_post`   | Post compression stress / yield (%)               |
| `Sag`      | Deflection of bottom under 8 kg (mm)              |

---

## 6. Design Guidelines

- **U_post ≈ 35–45%** → optimal material use and safety margin
- **U_rib ≥ 20%** → rib is active in supporting stack
- **Sag ≤ 2 mm** → perceptually flat, robot-ready
- **U_panel too high** means risk of permanent sag

---

## 7. How to Tune the Model

| Objective              | Change                            |
|------------------------|-----------------------------------|
| Lighter crate          | Lower `t_wall`, `t_skin`          |
| Cheaper crate          | Allow 60–70% stress utilisation   |
| More rigid floor       | Increase `h_rib` or reduce `pitch`|
| Deeper crate           | Raise `H_WALL`, allow more mass   |

---

## 8. Next step

> This model assumes simple mechanics and ideal support conditions.

> We can validate with finite element analysis (FEA).


In [44]:
#!/usr/bin/env python3
"""

Enumerate practical banana-fibre/HDPE returnable produce-crate geometries
and filter on

  • bottom-panel bending  (strip model between ribs)
  • rib compression / Euler buckle  (stack load through ribs)
  • corner-post compression         (stack load through posts)
  • global live-load deflection     (sag under produce only)
  • ergonomic stack height          (≤ STACK_MAX crates)
  • empty-crate mass window         (≈1.5–3 kg for 600 × 400 mm pool)

Material & service assumptions
------------------------------
* Banana-fibre / HDPE composite, 15 vol-% fibre, 10 % foamed voids  
    – E  = 1.2 GPa,  σ_y = 18 MPa,  ρ = 850 kg m⁻³
* 600 × 400 mm footprint, 220 mm inner height, six-high ergonomic limit
* Corner-stacking crate (column loads borne mainly by corner posts)
* Bottom panel simply supported on ribs (strip span = rib pitch)

Author: Ahmed Elhefnawy
Date  : 24-May-2025
"""

import numpy as np
from itertools import product

# ── material ───────────────────────────────────────────────────
E       = 1.2e9      # Pa   – tensile modulus
SIGMA_Y = 18e6       # Pa   – 0.2 % offset yield (flexural)
RHO     = 850        # kg/m³ – foamed banana-HDPE composite

# ── crate footprint & service loads ────────────────────────────
L, W   = 0.60, 0.40        # m   – footprint
H_WALL = 0.22              # m   – inner wall height
M_LOAD = 8.0               # kg  – produce per crate
G      = 9.81              # m/s²
STACK_MAX   = 6            # crates in ergonomic column
DEFLECT_LIM = 0.003        # m   – live-load sag limit (3 mm)
MASS_MIN, MASS_MAX = 1.5, 3.0   # kg – practical empty-mass window

# ── search-grid definitions ──────────────────
t_skin = np.arange(0.0015, 0.0031, 0.0003)   # 1.5…3.0 mm bottom skin
t_wall = np.arange(0.003 , 0.0046, 0.0005)   # 3.0…4.5 mm walls/posts
pitch  = np.arange(0.07  , 0.11 , 0.01  )    # 70…100 mm rib pitch
h_rib  = np.arange(0.015 , 0.026 , 0.003 )   # 15…24 mm rib height
t_rib  = 0.003                               # 3 mm rib-web (fixed)

# ── helper functions ───────────────────────────────────────────
def bottom_panel_metrics(ts: float, span: float, q: float):
    """Return (sigma_panel, delta_panel) for a simply-supported strip."""
    M     = q * span**2 / 8
    sigma = 6 * M / ts**2
    delta = 5 * q * span**4 / (384 * E * ts**3)
    return sigma, delta

def rib_metrics(h: float, n_stack: int, n_ribs: int):
    """Return rib compressive stress & Euler-buckling stress."""
    P_tot      = (n_stack - 1) * M_LOAD * G      # N  – full stack
    P_each     = P_tot / n_ribs                  # N  – per rib
    sigma_comp = P_each / (t_rib * h)
    slender    = h / t_rib
    sigma_eul  = (np.pi ** 2) * E / slender**2
    return sigma_comp, sigma_eul

def post_stress(tw: float, n_stack: int, m_crate: float):
    """Return compressive stress in a corner post."""
    A     = tw ** 2
    P     = (n_stack - 1) * (M_LOAD + m_crate) * G / 4
    return P / A

# ── design-space sweep ─────────────────────────────────────────
results = []
for ts, tw, p, h in product(t_skin, t_wall, pitch, h_rib):

    # rib counts (ribs parallel to length & width respectively)
    n_len = max(int(np.floor(L / p)) - 1, 1)
    n_wid = max(int(np.floor(W / p)) - 1, 1)
    n_ribs = n_len + n_wid

    # bottom panel: live-load only
    q_live = (M_LOAD * G) / (L * W)
    sigma_p, delta_p = bottom_panel_metrics(ts, p, q_live)
    if sigma_p > 0.85 * SIGMA_Y or delta_p > DEFLECT_LIM:
        continue

    # rib compression + Euler check (full stack)
    sigma_r, sigma_eul = rib_metrics(h, STACK_MAX, n_ribs)
    if sigma_r > 0.5 * SIGMA_Y or sigma_r > 0.8 * sigma_eul:
        continue

    # mass estimate (kg)
    vol_skin  = L * W * ts
    vol_ribs  = (n_len * W + n_wid * L) * h * t_rib
    vol_walls = 2 * (L + W) * H_WALL * tw
    vol_posts = 4 * H_WALL * tw * tw
    m = RHO * (vol_skin + vol_ribs + vol_walls + vol_posts)

    if not (MASS_MIN <= m <= MASS_MAX):
        continue

    # corner-post compression
    sigma_post = post_stress(tw, STACK_MAX, m)
    if sigma_post > 0.4 * SIGMA_Y:
        continue

    # store feasible design with all metrics
    results.append(dict(
        N=STACK_MAX, ts=ts, tw=tw, pitch=p, h_rib=h,
        mass=m,
        util_panel=sigma_p / SIGMA_Y,
        util_rib=sigma_r / SIGMA_Y,
        util_post=sigma_post / SIGMA_Y,
        sag=delta_p * 1e3  # to mm
    ))

# ── report ─────────────────────────────────────────────────────
if not results:
    print("No feasible design – adjust limits or widen grids.")
    exit()

# sort lightest first
results.sort(key=lambda r: r["mass"])

# print header
print(
    " N  ts[mm] tw[mm] pitch h_rib  Mass[kg]"
    "  U_panel  U_rib  U_post  Sag[mm]"
)

# print up to 40 best designs
for r in results[:40]:
    print(
        f"{r['N']:2d}"
        f" {r['ts']*1e3:6.1f}"
        f" {r['tw']*1e3:6.1f}"
        f"  {r['pitch']*1e3:4.0f}"
        f"  {r['h_rib']*1e3:4.0f}"
        f"  {r['mass']:6.2f}"
        f"   {r['util_panel']*100:5.1f}%"
        f"  {r['util_rib']*100:5.1f}%"
        f"  {r['util_post']*100:6.1f}%"
        f"   {r['sag']:6.2f}"
    )


 N  ts[mm] tw[mm] pitch h_rib  Mass[kg]  U_panel  U_rib  U_post  Sag[mm]
 6    1.5    4.5    90    15    2.15     4.9%    6.1%    34.1%     0.07
 6    1.5    4.5   100    15    2.15     6.1%    6.1%    34.1%     0.11
 6    1.5    4.5    90    18    2.18     4.9%    5.0%    34.2%     0.07
 6    1.5    4.5   100    18    2.18     6.1%    5.0%    34.2%     0.11
 6    1.5    4.5    80    15    2.19     3.9%    4.8%    34.3%     0.04
 6    1.5    4.5    70    15    2.20     3.0%    4.4%    34.3%     0.03
 6    1.5    4.5    90    21    2.21     4.9%    4.3%    34.3%     0.07
 6    1.5    4.5   100    21    2.21     6.1%    4.3%    34.3%     0.11
 6    1.8    4.5    90    15    2.21     3.4%    6.1%    34.4%     0.04
 6    1.8    4.5   100    15    2.21     4.2%    6.1%    34.4%     0.06
 6    1.5    4.5    80    18    2.22     3.9%    4.0%    34.4%     0.04
 6    1.5    4.5    90    24    2.24     4.9%    3.8%    34.4%     0.07
 6    1.5    4.5   100    24    2.24     6.1%    3.8%    34.4% 

## Best Overall Crate Design

| Parameter      | Value       | Description                                |
|----------------|-------------|--------------------------------------------|
| `N`            | 6           | Max stack height (crates)                  |
| `ts`           | 1.8 mm      | Bottom panel thickness                     |
| `tw`           | 4.5 mm      | Wall and corner post thickness             |
| `pitch`        | 70 mm       | Rib spacing (center-to-center)             |
| `h_rib`        | 15 mm       | Rib height                                 |
| `Mass`         | 2.26 kg     | Total crate mass                           |
| `U_panel`      | 2.1%        | Bottom panel stress as % of yield stress   |
| `U_rib`        | 4.4%        | Rib compression stress as % of yield       |
| `U_post`       | 34.5%       | Corner post stress as % of yield           |
| `Sag`          | 0.01 mm     | Panel deflection under 8 kg produce load   |

---

### Interpretation

- **Very low sag** → stiff, visually flat bottom
- **Good structural utilization** (especially `U_post`)
- **Low mass** → easy to handle, cost-efficient
- **Rib design** contributes meaningfully without overuse



## Stiff Crate Design 


| Parameter      | Value       | Description                                |
|----------------|-------------|--------------------------------------------|
| `N`            | 6           | Max stack height (crates)                  |
| `ts`           | 1.8 mm      | Bottom panel thickness                     |
| `tw`           | 4.5 mm      | Wall and corner post thickness             |
| `pitch`        | 70 mm       | Rib spacing (center-to-center)             |
| `h_rib`        | 21 mm       | Rib height                                 |
| `Mass`         | 2.34 kg     | Total crate mass                           |
| `U_panel`      | 2.1%        | Bottom panel stress as % of yield stress   |
| `U_rib`        | 3.1%        | Rib compression stress as % of yield       |
| `U_post`       | 34.8%       | Corner post stress as % of yield           |
| `Sag`          | 0.01 mm     | Panel deflection under 8 kg produce load   |

---

### Interpretation

- **Exceptionally low sag** → ideal for robotic flatness and rigidity
- **High post utilization** → safe and efficient under stacking
- **Ribs work conservatively** → more safety margin or nesting flexibility
- Slightly heavier than the lightest design, but with much higher base stiffness
